In [1]:
# !pip install langchain chromadb google-cloud-aiplatform unstructured jq langchain-community langchain-google-vertexai

In [2]:
import json
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain_community.vectorstores import Chroma
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
import vertexai

ModuleNotFoundError: No module named 'langchain'

In [ ]:
PROJECT_ID = !gcloud config get-value project  # noqa: E999
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
with open("tomato_diseases.json", "r") as f:
    json_data = json.load(f)

documents = []  # List to store Langchain documents

for disease, treatment in json_data.items():
    document = Document(
        page_content=f"{disease}: {treatment}"
    )  # Combine disease and treatment
    documents.append(document)

In [ ]:
embedding_function = VertexAIEmbeddings(
    model_name="text-embedding-004"
)
db = Chroma.from_documents(documents, embedding_function)

In [ ]:
query = "How to handle Bacterial Spot?"
docs = db.similarity_search(query, k=1)
print(docs[0].page_content)

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)


llm = VertexAI(model_name="text-bison@001", max_output_tokens=1024)

# q/a chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [ ]:
def ask_question(question: str):
    prompt_template = f"""
    You are an expert in agricultural diseases. Please provide a detailed, easy-to-understand explanation for the following question:
    
    {question}
    
    Your response should be natural and conversational.
    """
    response = qa.invoke({"query": prompt_template})
    print(f"Response: {response['result']}\n")

In [ ]:
ask_question("How to handle Late Blight?")